Cleaning the countypres_2000-2024 file 
1. nettoyage du 'bruit', on ne garde que Repliblicans & Democrates
2. on crée des % pour avoir le même poids pour tous les états (500 à LA ce n'est aps la même chose que 500 votes dans un village du texas)
3. reshaping : format long --> format large

In [3]:
import pandas as pd
import numpy as np

# ==========================================
# STEP 1: LOAD DATA
# ==========================================
# Load the raw CSV file using the correct separator
file_path = '/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/election_results/countypres_2000-2024.csv'
df_raw = pd.read_csv(file_path, sep=None, engine='python')

print("✅ Fichier chargé !")
print(f"Colonnes trouvées : {df_raw.columns.tolist()}")

# Display initial shape
print(f"Initial dataset shape: {df_raw.shape}")

# ==========================================
# STEP 2: DATA CLEANING & FILTERING
# ==========================================
# We only want to analyze the two major parties: Democrat and Republican.
# This filters out Green, Libertarian, and other minor parties.
df_clean = df_raw[df_raw['party'].isin(['DEMOCRAT', 'REPUBLICAN'])].copy()
print("✅ Filtrage réussi !")

# Calculate the 'vote_share': the percentage of total votes a candidate received.
# Formula: Candidate Votes / Total Votes in the County
df_clean['vote_share'] = df_clean['candidatevotes'] / df_clean['totalvotes']

# ==========================================
# STEP 3: RESHAPING (PIVOT)
# ==========================================
# Transform the data from "Long Format" (one row per candidate) 
# to "Wide Format" (one row per county per year).
# We want separate columns for Democrat and Republican vote shares.
df_pivot = df_clean.pivot_table(
    index=['year', 'state_po', 'county_name', 'county_fips'], # Unique identifiers for a row
    columns='party',                                          # Values to become new columns
    values='vote_share'                                       # The metric to populate the cells
).reset_index()

# Clean up column names (remove the hierarchy created by pivot_table)
df_pivot.columns.name = None

# Fill NaN values with 0 (in case a party received 0 votes in a county)
df_pivot = df_pivot.fillna(0)

# ==========================================
# STEP 4: EXPORT & VALIDATION
# ==========================================
# Save the processed file for the next steps (Modeling)
output_filename = "votes_cleaned_2000_2024.csv"
df_pivot.to_csv(output_filename, index=False)

print(f"Processing complete. Data saved to '{output_filename}'")
print("\nFirst 5 rows of the cleaned dataset:")
print(df_pivot.head())

✅ Fichier chargé !
Colonnes trouvées : ['year', 'state', 'state_po', 'county_name', 'county_fips', 'office', 'candidate', 'party', 'candidatevotes', 'totalvotes', 'version', 'mode']
Initial dataset shape: (94409, 12)
✅ Filtrage réussi !
Processing complete. Data saved to 'votes_cleaned_2000_2024.csv'

First 5 rows of the cleaned dataset:
   year state_po  county_name  county_fips  DEMOCRAT  REPUBLICAN
0  2000       AK   DISTRICT 1       2001.0  0.192909    0.703275
1  2000       AK  DISTRICT 10       2010.0  0.246961    0.638564
2  2000       AK  DISTRICT 11       2011.0  0.292693    0.567335
3  2000       AK  DISTRICT 12       2012.0  0.269833    0.608604
4  2000       AK  DISTRICT 13       2013.0  0.335012    0.485081


Cleaning file DP03_economic.csv : 
1. on ne garde que 4 variables pour éviter l'Overfitting (chômage, pauvreté, revenus, fonctionnaires)

In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# STEP 1: LOAD DATA
# ==========================================
file_path_eco = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet/DP03_economics.xlsx - DP03_economic.csv"

# Load with header=1 to get the codes
df_eco = pd.read_csv(file_path_eco, header=1)
df_eco = df_eco.iloc[1:] # Drop description row

# ==========================================
# STEP 2: SELECT VARIABLES (CUSTOMIZED)
# ==========================================
# I added 'DP03_0043PE' which is "Government workers" (Percent)
# You can add as many as you want here if you know the code!

columns_mapping = {
    'GEO_ID': 'fips',
    'NAME': 'county_name',
    'DP03_0009PE': 'unemployment_rate',      # Chômage
    'DP03_0062E': 'median_income',           # Revenu
    'DP03_0128PE': 'poverty_rate',           # Pauvreté
    'DP03_0043PE': 'public_workers_pct'      # (BONUS) Fonctionnaires
}

# Keep and rename
df_eco_clean = df_eco[columns_mapping.keys()].rename(columns=columns_mapping).copy()

# ==========================================
# STEP 3: CLEANING & TYPES
# ==========================================
# Convert all value columns to numbers
cols_to_convert = ['unemployment_rate', 'median_income', 'poverty_rate', 'public_workers_pct']

for col in cols_to_convert:
    df_eco_clean[col] = pd.to_numeric(df_eco_clean[col], errors='coerce')

# Clean FIPS (Keep last 5 digits)
df_eco_clean['fips'] = df_eco_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
df_eco_clean.to_csv("economics_cleaned_2023.csv", index=False)

print("✅ Updated economic file with 4 variables!")
print(df_eco_clean.head())

Cleaning the DP02_socio.csv file :


In [ ]:
import pandas as pd

# ==========================================
# STEP 1: LOAD SOCIAL DATA (DP02)
# ==========================================
# ⚠️ Update the path below with your own "Copy Path" for DP02
file_path_socio = "/Users/jessicabourdouxhe/Desktop/Master 1/Data/Projet /elections-nlp-project/data/raw/socio-economic/DP02_socio.csv"

df_socio = pd.read_csv(file_path_socio, header=1)
df_socio = df_socio.iloc[1:] # Drop description row

print(f"✅ Social Data Loaded! Shape: {df_socio.shape}")

# ==========================================
# STEP 2: SELECT EDUCATION DATA
# ==========================================
# Key Variable:
# - DP02_0068PE: Percent of people (25+) with a Bachelor's degree or higher

columns_mapping_socio = {
    'GEO_ID': 'fips',
    'DP02_0068PE': 'education_bachelors_pct'
}

df_socio_clean = df_socio[columns_mapping_socio.keys()].rename(columns=columns_mapping_socio).copy()

# ==========================================
# STEP 3: CLEANING
# ==========================================
# Convert to numbers
df_socio_clean['education_bachelors_pct'] = pd.to_numeric(df_socio_clean['education_bachelors_pct'], errors='coerce')

# Clean FIPS code (last 5 characters)
df_socio_clean['fips'] = df_socio_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
output_filename_socio = "education_cleaned_2023.csv"
df_socio_clean.to_csv(output_filename_socio, index=False)

print(f"🎉 Process Complete! File saved: {output_filename_socio}")
print(df_socio_clean.head())

Cleaning the file DP05_demo.csv :
1. on garde la variable age 
2. on garde les variables relatives à l'origine éthnique

In [ ]:
import pandas as pd

# ==========================================
# STEP 1: LOAD DEMOGRAPHIC DATA (DP05)
# ==========================================
# ⚠️ Remplacez le chemin par celui de votre VRAI fichier DP05 sur votre ordinateur
file_path_demo = "TON_CHEMIN_VERS_DP05_demo.csv"

df_demo = pd.read_csv(file_path_demo, header=1)
df_demo = df_demo.iloc[1:] # On enlève la ligne de description

print(f"✅ Fichier Démo chargé ! Taille : {df_demo.shape}")

# ==========================================
# STEP 2: SELECT VARIABLES
# ==========================================
# Voici les variables clés pour la politique US :
# - DP05_0018E : Age Médian (Les jeunes votent plus à gauche, les âgés à droite)
# - DP05_0037PE : % Blancs (White alone)
# - DP05_0038PE : % Noirs (Black or African American alone)
# - DP05_0071PE : % Hispaniques (Hispanic or Latino)

columns_mapping_demo = {
    'GEO_ID': 'fips',
    'DP05_0018E': 'median_age',
    'DP05_0037PE': 'white_pct',
    'DP05_0038PE': 'black_pct',
    'DP05_0071PE': 'hispanic_pct'
}

# On sélectionne et renomme
df_demo_clean = df_demo[columns_mapping_demo.keys()].rename(columns=columns_mapping_demo).copy()

# ==========================================
# STEP 3: CLEANING
# ==========================================
# Conversion en nombres
cols_to_convert = ['median_age', 'white_pct', 'black_pct', 'hispanic_pct']

for col in cols_to_convert:
    df_demo_clean[col] = pd.to_numeric(df_demo_clean[col], errors='coerce')

# Nettoyage du code FIPS (les 5 derniers chiffres)
df_demo_clean['fips'] = df_demo_clean['fips'].astype(str).str[-5:]

# ==========================================
# STEP 4: SAVE
# ==========================================
output_filename_demo = "demographics_cleaned_2023.csv"
df_demo_clean.to_csv(output_filename_demo, index=False)

print(f"🎉 Terminé ! Fichier sauvegardé : {output_filename_demo}")
print(df_demo_clean.head())